In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd './drive/My Drive/CVPR 16682/src'

In [2]:
import torch
import numpy as np
import torch.nn as nn
import numpy as np
from scipy.stats import gaussian_kde
import os
from os.path import exists
import copy
import math

In [3]:
!pip install torchnet

In [ ]:
### For baseline method defaults
sparsity, mergedsparsity = 0.65, 0.3
clusternum, maxsubnets, unusedthreshold = 3, 5, 8

### For replay-only defaults
# clusternum, maxsubnets, unusedthreshold = 1, 1, 8
# sparsity, mergedsparsity = 0.001, 0.001

### For pruning-only defaults
# sparsity, mergedsparsity = 0.65, 0.3
# clusternum, maxsubnets, unusedthreshold = 3, 100, 100

lr=0.1
onlineepochs, trainepochs, ftepochs = 20,10,20

run_id = "run-000"

arch = "modresnet18"
# arch = "vgg16"

# dataset, finaltask = "online_cifar_rotmnist", 22
# dataset, finaltask = "online_cifar_jump20rotmnist", 19
# dataset, finaltask = "online_cifar_jump30rotmnist", 18
dataset, finaltask = "online_mixed_cifar_pmnist", 27

basepath = os.path.join("../checkpoints/", str(dataset), (str(sparsity) + "_" + str(mergedsparsity)), (str(clusternum) + '_' + str(maxsubnets) + "_" + str(unusedthreshold)), str(arch), str(run_id), str(lr))
finalpath = os.path.join(basepath,str(finaltask),"final.pt")

finalckpt = torch.load(finalpath, map_location=torch.device('cpu'))
print(basepath)

In [36]:
manager = finalckpt['manager']
evaldicts = manager.eval_dicts
olaccs,netcap,trtimes,oldtasks = evaldicts['onlineaccs'], evaldicts['overall_capacity'], evaldicts['training_times'], evaldicts['identifiedtasks']
newtasks,mergehist,idacchist,accthreshhist,tasksbyset = evaldicts['newdistributions'], evaldicts['mergehistory'], evaldicts['ID_ACCS_history'], evaldicts['ACCthresholds_history'], evaldicts['tasks_by_set']

In [37]:
print(manager.FLOPcounts)

{'inference': 430200815616, 'training': 122300006400000, 'retraining': 0}


Zip Task Identification History and Store Capacity, Runtime, and Accuracies

In [7]:
taskidbyset, bufferbyset, buffersizebyset, caps, totals,aves, times = {}, {}, {}, [], [], [], []
for key in olaccs.keys():
  if len(times) == 0:
    times.append(trtimes[key])
  else:
    times.append(trtimes[key]-trtimes[key-1])

  totals.append(torch.mean(torch.tensor(olaccs[key])))
  aves.append(torch.mean(torch.tensor(totals)))

  taskcap = []
  if len(list(netcap.keys())) > 0:
    for key2 in netcap[key]:
      taskcap.append(netcap[key][key2])
    caps.append(torch.tensor(1)-torch.mean(torch.tensor(taskcap)))

  if key in list(newtasks.keys()):
    taskidbyset[key] = newtasks[key]
  else:
    taskidbyset[key] = oldtasks[key]

In [95]:
### Dont use oracle for rotMNIST, since theres no definitive answer to whether a shift is a new task or not
taskIDoracle = None
rotID = {}
rotTasks = []

if dataset == "online_mixed_cifar_pmnist":
  if unusedthreshold == 8:
    taskIDoracle  = [0,1,2,3,4,5,6,7,8,9,1,2,12,4,5,15,7,8,18,1,2,21,4,5,24,7,8,27]
  elif unusedthreshold >= 30:
    taskIDoracle  = [0,1,2,3,4,5,6,7,8,9,1,2,12,4,5,15,7,8,18,1,2,21,4,5,24,7,8,27]

if dataset == "online_cifar_rotmnist":
  rotID = {0: (taskidbyset[2]), 1:(taskidbyset[5]), 2:(taskidbyset[8]), 3:(taskidbyset[11]), 4:(taskidbyset[14]), 5:(taskidbyset[17]), 6: (taskidbyset[20])}
  rotTasks = [2,5,8,11,14,17,20]
elif dataset == "online_cifar_jump20rotmnist":
  rotID = {0: (taskidbyset[2]), 1:-1, 2:(taskidbyset[7]), 3:-1, 4:(taskidbyset[12]), 5:-1, 6: (taskidbyset[17])}
  rotTasks = [2,-1,7,-1,12,-1,17]
elif dataset == "online_cifar_jump30rotmnist":
  rotID = {0: (taskidbyset[2]), 1:-1, 2:-1, 3:(taskidbyset[9]), 4:-1, 5:-1, 6: (taskidbyset[16])}
  rotTasks = [2,-1,-1,9,-1,-1,16]

In [ ]:
taskidbyset

Store the resulting processed dictionaries and lists

In [96]:
processedEvals = {'rotID':rotID, 'rotTasks':rotTasks, 'taskOracle':taskIDoracle, 'bufferbyset':bufferbyset,'buffersizebyset':buffersizebyset,'taskIDbyset':taskidbyset, 'capacity':caps, 'meanaccbyset':totals, 'runningmeanacc':aves, 'runtimes':times}

In [97]:
torch.save(processedEvals, (basepath + "/processedEval.pt"))

# Assess processed measures

In [47]:
print(buffersizebyset.keys())
print(buffersizebyset.values())

dict_keys([])
dict_values([])


In [99]:
for i in range(len(aves)):
  aves[i] = aves[i].tolist()
print(aves)

[41.98381805419922, 57.50537872314453, 62.368194580078125, 54.43927001953125, 57.06584548950195, 57.46219253540039, 55.144920349121094, 56.73997497558594, 57.14055633544922, 55.235015869140625, 56.538475036621094, 56.59322738647461, 55.304744720458984, 56.244510650634766, 56.619102478027344, 55.59595489501953, 56.13396072387695, 56.37455749511719, 55.543792724609375, 56.10847854614258, 56.48528289794922, 55.48893737792969, 56.11225128173828, 56.53421401977539, 55.603668212890625, 55.69459533691406, 56.310333251953125, 55.563316345214844]


In [90]:
for i in range(len(caps)):
  caps[i] = caps[i].tolist()
print(caps)

[0.6510937213897705, 0.421875, 0.27562499046325684, 0.18140625953674316, 0.11890625953674316, 0.3253124952316284, 0.21265625953674316, 0.13875001668930054, 0.34843748807907104, 0.22484374046325684, 0.22484374046325684, 0.22484374046325684, 0.14625000953674316, 0.32999998331069946, 0.32999998331069946, 0.2149999737739563, 0.2149999737739563, 0.140625, 0.32859373092651367, 0.32859373092651367, 0.32859373092651367, 0.2149999737739563, 0.2149999737739563, 0.2149999737739563, 0.140625, 0.140625, 0.140625, 0.34937500953674316]


In [91]:
print(times)

[8.353816032409668, 9.859452962875366, 9.368222713470459, 8.201166152954102, 9.511569261550903, 43.2906129360199, 8.27520489692688, 9.656780242919922, 68.54375004768372, 8.364223718643188, 33.52402377128601, 33.523252964019775, 8.43705940246582, 71.5902190208435, 7.254626989364624, 8.557373046875, 49.568071365356445, 10.02517557144165, 81.248943567276, 65.75788760185242, 65.58712148666382, 8.657265663146973, 65.7698724269867, 7.094752550125122, 8.732950448989868, 65.60110592842102, 65.87219476699829, 104.8391842842102]
